<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Probabilistic-Inference" data-toc-modified-id="Probabilistic-Inference-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Probabilistic Inference</a></span></li><li><span><a href="#Directories" data-toc-modified-id="Directories-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Directories</a></span></li><li><span><a href="#Reading-in-Data" data-toc-modified-id="Reading-in-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Reading in Data</a></span></li><li><span><a href="#Compute-Likelihood,-$L(D-|-sin(I_*))$" data-toc-modified-id="Compute-Likelihood,-$L(D-|-sin(I_*))$-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Compute Likelihood, $L(D | sin(I_*))$</a></span><ul class="toc-item"><li><span><a href="#Executing" data-toc-modified-id="Executing-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Executing</a></span></li></ul></li><li><span><a href="#Archive" data-toc-modified-id="Archive-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Archive</a></span></li></ul></div>

## Probabilistic Inference

$$\mathcal{L}(D | sin(I_*)) = \int_1^\infty u p_1(u) p_2(u\sin(I_*)) du$$

where $p_1$ is the probability distribution of $V=2\pi R_* / P_{rot}$ and $p_2$ is the probability distribution of $V \sin(I_*)$

In [1]:
import scipy.stats as stats
import pandas as pd
import numpy as np
from scipy.integrate import quad

In [2]:
import plotly.plotly  as py
import plotly.graph_objs as go

## Directories

In [3]:
main_dir = "/Volumes/LaCie/Research/NAC/"
data_dir = main_dir+"data/"
out_dir = main_dir+"output/"

## Reading in Data

In [4]:
df = pd.read_csv("{}MN_17_3616_point.csv".format(data_dir),
                index_col=[0])

In [5]:
df_trim = df[df.V.notnull()].reset_index(drop=True)
df_trim = df_trim[np.isfinite(df_trim.EVSINI)].reset_index(drop=True)

## Compute Likelihood, $L(D | sin(I_*))$

In [6]:
from multiprocessing import Pool, Manager, active_children

## list of columns to retrive
features = ["V", "V_sigma", "VSINI", "EVSINI"]

def integrand(u, p1, p2, sini):
    r'''
    Parameters:
    -----------
    u : For use in scipy.integrate.quad
    p1 : scipy.stats._distn_infrastructure.rv_frozen
        Frozen distribution of "v"
    p2 : scipy.stats._distn_infrastructure.rv_frozen
        Frozen distribution of "vsini"
    sini : float
        sin(I_*) measurement for a star
    
    Returns:
    --------
    u * (p1 evaluated at u) * (p2 evaluated at u*sini)
    '''
    return u*p1.pdf(u)*p2.pdf(u*sini)

def compute_L(args, start=0.0, end=300.0):
    global integrand
    sini, p1, p2 = args
    return quad(integrand, start, end, args=(p1,p2, sini))[0]

pool = Pool(8)

def likelihood(sinis, v, v_sigma, vsini, vsini_sigma):
    r'''
    Computes integrand from 0 to +infinity
    
    Parameters:
    -----------
    sinis : array or list
        sinis to compute the likelihood for
    v : float
        velocity for a star
    v_sigma : float
        Error on the velocity
    vsini : float
        vsini for a star
    vsini_sigma : float
        error on vsini

    Returns:
    --------
    (pdf/norm) : ndarray
        Normalized likelihoods given a range of sini
    '''
    
    p1 = stats.norm(loc=v, scale=v_sigma)
    p2 = stats.norm(loc=vsini, scale=vsini_sigma)
    
    pdf = pool.map(compute_L, [(i, p1, p2) for i in sinis])
    #pdf = np.array([quad(integrand, 0, np.inf, args=(p1,p2, sini)) for sini in sinis])[:,0]
    
    norm = np.trapz(pdf, sinis, dx=0.01)
    
    return pdf/norm

### Executing

In [7]:
Ls = Manager().list()
#traces = Manager().list()

def main(index):
    global Ls
    global traces
    global df_trim
    global sini
    
    if index % 10 == 0:
        print("Inspecting Star {}...".format(index))
    
    v, v_sigma, vsini, vsini_sigma = df_trim.loc[index][features].values
    
    L = likelihood(sini, v, v_sigma, vsini, vsini_sigma)
    
    Ls.append(L)
    
#     traces = go.Scatter(
#             x = sini,
#             y = L,
#             name = df_trim.loc[index].Name)
    
#     Ls.append([L, traces])


In [8]:
from joblib import Parallel, delayed
sini = np.arange(0.0, 1.0, 0.002)

if __name__ == "__main__":
    Parallel(n_jobs=3, backend="threading")(
        map(delayed(main), range(len(df_trim))))
    
pool.join()

Inspecting Star 0...
Inspecting Star 10...
Inspecting Star 20...
Inspecting Star 30...
Inspecting Star 40...
Inspecting Star 50...


In [9]:
traces = []

for L in range(len(Ls)):
    traces.append(go.Scatter(
            x = sini,
            y = Ls[L],
            name = df_trim.loc[L].Name
        )
    )

In [10]:
layout = go.Layout(
            xaxis={
                "title" : "$\sin(I_*)$",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1,
                "range" : [0, 1]
                },
            yaxis={
                "title" : "$L$",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1
                },
        )
figure = go.Figure(data=traces, layout=layout)
py.iplot(figure, filename="psini")

In [11]:
temp = pd.DataFrame(list(Ls), columns = sini)
temp["Name"] = df_trim.Name
temp.to_csv(out_dir+"sinis.csv")

# df_sinis = pd.DataFrame([])

# traces = []
# for star in range(len(df_trim)):
    
#     if star % 10 == 0:
#         print("Inspecting Star {}...".format(star))
    
#     v, v_sigma, vsini, vsini_sigma = df_trim.loc[star][features].values
    
#     L = likelihood(sini, v, v_sigma, vsini, vsini_sigma)
    
#     df_sinis = df_sinis.append(pd.DataFrame(L))
    
#     traces.append(go.Scatter(
#         x = sini,
#         y = L,
#         name = df_trim.loc[star].Name))
    

In [ ]:
layout = go.Layout(
            xaxis={
                "title" : "$\sin(I_*)$",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1,
                "range" : [0, 1]
                },
            yaxis={
                "title" : "$L$",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1
                },
        )

figure = go.Figure(data=traces, layout=layout)
py.iplot(figure, filename="test")

## Archive

In [ ]:
def plot_likelihood(frame):
    v, v_sigma, vsini, vsini_sigma, _ = frame[features].values
    p1 = stats.norm(loc=v, scale=v_sigma).rvs(size=1000)
    p2 = stats.norm(loc=vsini, scale=vsini_sigma).rvs(size=1000)
    
    return [
        go.Histogram(
            x = v * p1 * p2,
            name = frame.Name,
            opacity = 0.6,
            xbins = {"size" : 1000}
            ),
        go.Histogram(
            x = p1,
            name = frame.Name,
            opacity = 0.6,
            xbins = {"size" : 1000}
            ),
        go.Histogram(
            x = p2,
            name = frame.Name,
            opacity = 0.6,
            xbins = {"size" : 1000}
            ),
           ]

In [ ]:
data = np.array([plot_likelihood(df_trim.loc[i]) for i in range(len(df_trim))])

layout = go.Layout(barmode='overlay',
            xaxis={
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1},
              yaxis={
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1}
                  
                  )

figure = go.Figure(data=list(data[:,0]), layout=layout)
py.iplot(figure, filename="likelihood")

In [ ]:
layout = go.Layout(barmode='overlay',
              xaxis={
                "title" : "p(V)",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1},
              yaxis={
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1}
                  )

figure = go.Figure(data=list(data[:,1]), layout=layout)
py.iplot(figure, filename="p1")

In [ ]:
layout = go.Layout(barmode='overlay',
              xaxis={
                "title" : "p(VSINI)",
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1},
              yaxis={
                "mirror" : "ticks",
                "linecolor" : "black",
                "linewidth" : 1}
                  )

figure = go.Figure(data=list(data[:,2]), layout=layout)
py.iplot(figure, filename="p2")